In [118]:
from models import FCC, Siamese, Siamese_no_sharing, Siamese_ws_auxilary
import dlc_practical_prologue
import torch
from torch import nn
from torch import optim
import torchvision
from torch.nn import functional as F
from tqdm import trange
import matplotlib.pyplot as plt
import models

In [2]:
train_input, train_target, train_classes, test_input, test_target, test_classes = dlc_practical_prologue.generate_pair_sets(1000)

In [42]:
# Weight-sharing "Siamese" LeNet
class Siamese(nn.Module):

    def __init__(self):
        super(Siamese, self).__init__()
        
        self.LeNet1 = nn.Sequential(
            nn.Conv2d(1,16,5),  # 16x10x10 (input is 1x14x14)
            nn.MaxPool2d(2),    # 16x5x5
            nn.ReLU(),
            nn.Conv2d(16,32,2), # 32x4x4
            nn.MaxPool2d(2),    # 32x2x2 (-> 1x128 before LeNet2)
            nn.ReLU()
        )
        self.LeNet2 = nn.Sequential(
            nn.Linear(128,64),  # 1x64
            nn.ReLU(),
            nn.Linear(64,32),   # 1x32
            nn.ReLU()
        )
        self.LeNet3 = nn.Sequential(
            nn.Linear(32,16),   # 1x16
            nn.Sigmoid(),
            nn.Linear(16,2)     # 1x2
        )
        
    def forward_bro(self, x):
        x = self.LeNet1(x)
        x = x.view(-1,1,128)
        x = self.LeNet2(x)
        return x
    
    def forward(self, x1, x2):
        x1 = self.forward_bro(x1)
        x2 = self.forward_bro(x2)
        x3 = x2 - x1
        x3 = self.LeNet3(x3)
        return x3

In [75]:
class Siamese_no_sharing(nn.Module):

    def __init__(self):
        super(Siamese_no_sharing, self).__init__()
        
        self.LeNet1_x1 = nn.Sequential(
            nn.Conv2d(1,16,5),  # 16x10x10 (input is 1x14x14)
            nn.MaxPool2d(2),    # 16x5x5
            nn.ReLU(),
            nn.Conv2d(16,32,2), # 32x4x4
            nn.MaxPool2d(2),    # 32x2x2 (-> 1x128 before LeNet2)
            nn.ReLU()
        )
        self.LeNet2_x1 = nn.Sequential(
            nn.Linear(128,64),  # 1x64
            nn.ReLU(),
            nn.Linear(64,32),   # 1x32
            nn.ReLU()
        )        
        
        self.LeNet1_x2 = nn.Sequential(
            nn.Conv2d(1,16,5),  # 16x10x10 (input is 1x14x14)
            nn.MaxPool2d(2),    # 16x5x5
            nn.ReLU(),
            nn.Conv2d(16,32,2), # 32x4x4
            nn.MaxPool2d(2),    # 32x2x2 (-> 1x128 before LeNet2)
            nn.ReLU()
        )
        self.LeNet2_x2 = nn.Sequential(
            nn.Linear(128,64),  # 1x64
            nn.ReLU(),
            nn.Linear(64,32),   # 1x32
            nn.ReLU()
        )
        
        self.LeNet3 = nn.Sequential(
            nn.Linear(32,16),   # 1x16
            nn.Sigmoid(),
            nn.Linear(16,2)     # 1x2
        )
        
    def forward_x2(self, x):
        x = self.LeNet1_x2(x)
        x = x.view(-1,1,128)
        x = self.LeNet2_x2(x)
        return x  
        
        
    def forward_x1(self, x):
        x = self.LeNet1_x1(x)
        x = x.view(-1,1,128)
        x = self.LeNet2_x1(x)
        return x
    
    def forward(self, x1, x2):
        x1 = self.forward_x1(x1)
        x2 = self.forward_x2(x2)
        x3 = x2 - x1
        x3 = self.LeNet3(x3)
        return x3

In [72]:
def train_model(model, train_input, train_target, train_classes = None, batch_size, nb_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)

    for e in range(nb_epochs):
        acc_loss = 0
        for b in range(0, train_input.size(0), batch_size):
            imgs = train_input.narrow(0, b, batch_size)
            imgs1 = imgs[:,0].view(batch_size, 1, 14, 14)
            imgs2 = imgs[:,1].view(batch_size, 1, 14, 14)
            if train_classes==None:
                output = model(imgs1, imgs2).view(batch_size, -1)
                loss = criterion(output, train_target.narrow(0, b, batch_size))
            else:
                x1_pred, x2_pred , output = model(imgs1, imgs2)
                x1_pred = x1_pred.view(batch_size, -1)
                x2_pred = x2_pred.view(batch_size, -1)
                output = output.view(batch_size, -1)
                loss = criterion(output, train_target.narrow(0, b, batch_size))
                loss_aux1 = aux_criterion(x1_pred,train_classes.narrow(0, b, batch_size)[:,0])
                loss_aux1 += aux_criterion(x2_pred,train_classes.narrow(0, b, batch_size)[:,1])
                loss = loss + loss_aux1
            
            acc_loss += loss.item()
            model.zero_grad()
            loss.backward()
            optimizer.step()
        #print(e, acc_loss)

In [7]:
def compute_nb_errors(model, input_data, target_data, batch_size):
    nb_errors = 0
    
    for b in range(0, input_data.size(0), batch_size):
            imgs = input_data.narrow(0, b, batch_size)
            target = target_data.narrow(0, b, batch_size)
            imgs1 = imgs[:,0].view(batch_size, 1, 14, 14)
            imgs2 = imgs[:,1].view(batch_size, 1, 14, 14)
            output = model(imgs1, imgs2).view(batch_size, -1)
            pred = output.max(1)[1]
            nb_errors += (pred-target).abs().sum().item()
    
    return nb_errors

In [ ]:
def error_rates(model, input_data, target_data, class_data=None, batch_size, rounds):
    error_rates = torch.empty(rounds)
    for r in trange(rounds):
        model = model
        

In [94]:
error_rates_ws = torch.empty(10)
for r in trange(10):
    model = Siamese()
    train_model(model, train_input, train_target, 100, 25)
    error_rates_ws[r] = compute_nb_errors(model, test_input, test_target, 100)/test_input.size(0)
error_rates_ws = torch.tensor(error_rates_ws)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:39<00:00,  3.92s/it]


In [97]:
error_rates_nows = torch.empty(10)
for r in trange(10):
    model = Siamese_no_sharing()
    train_model(model, train_input, train_target, 100, 25)
    error_rates_nows[r] = compute_nb_errors(model, test_input, test_target, 100)/test_input.size(0)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:38<00:00,  3.90s/it]


In [124]:
model = Siamese_ws_auxilary()

TypeError: super(type, obj): obj must be an instance or subtype of type